# Create Function to load data

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.optimize import curve_fit

In [ ]:
file_name = r""
independent_variable_name = r""
dependent_variable_name = r""

df =